In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/global-data-on-sustainable-energy/global-data-on-sustainable-energy (1).csv


In [2]:
# For Data Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# For Data Visualization

import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
import plotly.offline as pyo

from plotly.subplots import make_subplots
from plotly.offline import plot

# For Additional Use
import datetime
import calendar

# Template
pio.templates.default = "plotly_dark"

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('/kaggle/input/global-data-on-sustainable-energy/global-data-on-sustainable-energy (1).csv')
data.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.93911,67.709953


**Data Preparation**

Observation on Null Data

In [4]:
nulldata = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)[:25])
nulldata = nulldata.rename(columns = {0 : 'Null Values'})
nulldata

,Null Values
Renewables (% equivalent primary energy),2137
Financial flows to developing countries (US $),2089
Renewable-electricity-generating-capacity-per-capita,931
Value_co2_emissions_kt_by_country,428
gdp_growth,317
gdp_per_capita,282
Energy intensity level of primary energy (MJ/$2017 PPP GDP),207
Renewable energy share in the total final energy consumption (%),194
Access to clean fuels for cooking,169
Electricity from nuclear (TWh),126


In [5]:
fig_null_data = go.Figure()
fig_null_data.add_trace(go.Bar(
    x = nulldata.index,
    y = nulldata['Null Values'],
    text = nulldata['Null Values'],
    textposition = 'auto'
))
fig_null_data.update_layout(
    title = 'Null Values Count',
    xaxis_tickfont_size = 7,
    template = 'plotly_dark',
)
fig_null_data.show()

**Data Selection**

The analysis will focus on Sout East Asia, a region that consist of :
<ol>
<li>Brunei</li>
<li>Cambodia</li>
<li>Indonesia</li>
<li>Laos</li>
<li>Malaysia</li>
<li>Myanmar</li>
<li>Philippines</li>
<li>Singapore</li>
<li>Thailand</li>
<li>Vietnam</li>
</ol>
However, data for several countries is not available, hence being excluded. The following is the list of available countries for the region.
<ol>
<li>Cambodia</li>
<li>Indonesia</li>
<li>Malaysia</li>
<li>Myanmar</li>
<li>Philippines</li>
<li>Singapore</li>
<li>Thailand</li>
</ol>


In [6]:
sea = ['Cambodia','Indonesia','Malaysia','Myanmar','Philippines','Singapore','Thailand']

In [7]:
data_sea = data[data['Entity'].isin(sea)]
data_sea.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
567,Cambodia,2000,16.600000,3.6,0.83,NaN,81.58,0.42,0.0,0.05,10.638298,739.45550,7.91,1960.000000,NaN,9.993580,300.613679,95,181035.0,12.565679,104.990963
568,Cambodia,2001,14.624778,4.1,0.82,NaN,80.51,0.48,0.0,0.03,5.882353,769.40970,7.51,2150.000000,NaN,8.148386,321.150224,95,181035.0,12.565679,104.990963
569,Cambodia,2002,18.187100,4.7,0.97,NaN,80.97,0.66,0.0,0.03,4.347826,754.14470,7.05,2210.000038,NaN,6.578940,338.987477,95,181035.0,12.565679,104.990963
570,Cambodia,2003,19.300000,5.3,0.96,4630000.0,79.92,0.67,0.0,0.04,5.633803,803.81635,6.69,2380.000114,NaN,8.505896,362.335482,95,181035.0,12.565679,104.990963
571,Cambodia,2004,25.298384,5.9,1.02,409150000.0,80.69,0.73,0.0,0.03,3.947368,813.56445,6.23,2380.000114,NaN,10.340529,408.513639,95,181035.0,12.565679,104.990963


In [8]:
data_sea.shape

(147, 21)

In [9]:
sea_null = pd.DataFrame( data_sea.isnull().sum().sort_values(ascending=False)[:25])
sea_null = sea_null.rename(columns = {0 : 'Null Values'})
sea_null


,Null Values
Financial flows to developing countries (US $),45
Electricity from nuclear (TWh),42
Renewables (% equivalent primary energy),42
Energy intensity level of primary energy (MJ/$2017 PPP GDP),7
Renewable energy share in the total final energy consumption (%),7
Value_co2_emissions_kt_by_country,7
Entity,0
Latitude,0
Land Area(Km2),0
Density\n(P/Km2),0


In [10]:
fig_null_data_sea = go.Figure()
fig_null_data_sea.add_trace(go.Bar(
    x = sea_null.index,
    y = sea_null['Null Values'],
    text = sea_null['Null Values'],
    textposition = 'auto'
    
))
fig_null_data_sea.update_layout(
    title = 'Null Values Count',
    xaxis_tickfont_size = 7,
    template = 'plotly_dark',
)

# **Introduction**

Southeast Asia is one of the fastest-growing regions in the world, and its energy demand is expected to continue to grow in the coming decades. This growth is being driven by a number of factors, including population growth, economic development, and urbanization.

The region's primary energy mix is dominated by fossil fuels, with coal, oil, and natural gas accounting for over 80% of total consumption. This reliance on fossil fuels has led to high levels of CO2 emissions, which contribute to climate change.

In recent years, there has been growing awareness of the need to reduce CO2 emissions in Southeast Asia. Governments across the region have set ambitious targets for renewable energy deployment, and there is a growing investment in energy efficiency measures.

This analysis will examine the trends in electricity consumption,primary energy consumption and CO2 emissions in Southeast Asia. 


# **Analysis Coverage**

The analysis will focus on Sout East Asia, a region that consist of :
<ol>
<li>Brunei</li>
<li>Cambodia</li>
<li>Indonesia</li>
<li>Laos</li>
<li>Malaysia</li>
<li>Myanmar</li>
<li>Philippines</li>
<li>Singapore</li>
<li>Thailand</li>
<li>Vietnam</li>
</ol>
However, data for several countries is not available, hence being excluded. The following is the list of available countries for the region.
<ol>
<li>Cambodia</li>
<li>Indonesia</li>
<li>Malaysia</li>
<li>Myanmar</li>
<li>Philippines</li>
<li>Singapore</li>
<li>Thailand</li>
</ol>


# **Analysis on Electricity Consumption**

In [11]:
electricity_sea = data_sea[['Entity','Year','Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)','Electricity from renewables (TWh)',]]
electricity_sea.head()

,Entity,Year,Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh)
567,Cambodia,2000,0.42,0.0,0.05
568,Cambodia,2001,0.48,0.0,0.03
569,Cambodia,2002,0.66,0.0,0.03
570,Cambodia,2003,0.67,0.0,0.04
571,Cambodia,2004,0.73,0.0,0.03


In [12]:
electricity_sea_sum = electricity_sea.groupby('Year').sum()
electricity_sea_sum['Total Electricity'] = electricity_sea_sum.sum(axis=1)
electricity_sea_sum['% Fossil Fuel'] = (electricity_sea_sum['Electricity from fossil fuels (TWh)']/electricity_sea_sum['Total Electricity'])*100
electricity_sea_sum['% Nuclear'] = (electricity_sea_sum['Electricity from nuclear (TWh)']/electricity_sea_sum['Total Electricity'])*100
electricity_sea_sum['% Renewable'] = (electricity_sea_sum['Electricity from renewables (TWh)']/electricity_sea_sum['Total Electricity'])*100
electricity_sea_sum.head()

,Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Total Electricity,% Fossil Fuel,% Nuclear,% Renewable
Year,,,,,,,
2000,281.00,0.0,55.07,336.07,83.613533,0.0,16.386467
2001,305.34,0.0,55.17,360.51,84.696680,0.0,15.303320
2002,326.35,0.0,54.58,380.93,85.671908,0.0,14.328092
2003,346.20,0.0,53.89,400.09,86.530531,0.0,13.469469
2004,376.47,0.0,56.20,432.67,87.010886,0.0,12.989114


## **Electricity Consumption & Source of Electricity Mix**

In [13]:
fig_electricity_sea_volume = go.Figure()
fig_electricity_sea_volume.add_trace(go.Scatter(x = electricity_sea_sum.index, y = electricity_sea_sum['Total Electricity'], name = 'Total Electricity', mode = 'lines+markers'))
fig_electricity_sea_volume.add_trace(go.Scatter(x = electricity_sea_sum.index, y = electricity_sea_sum['Electricity from fossil fuels (TWh)'], name = ' Fossil Fuel', mode = 'lines+markers'))
fig_electricity_sea_volume.add_trace(go.Scatter(x = electricity_sea_sum.index, y = electricity_sea_sum['Electricity from nuclear (TWh)'], name = 'Nuclear', mode = 'lines+markers'))
fig_electricity_sea_volume.add_trace(go.Scatter(x = electricity_sea_sum.index, y = electricity_sea_sum['Electricity from renewables (TWh)'], name = 'Renewable', mode = 'lines+markers'))
fig_electricity_sea_volume.update_layout( title = 'Electricity Production in SEA', xaxis_tickfont_size = 7, template = 'plotly_dark')
fig_electricity_sea_volume.update_xaxes(rangeslider_visible = True)
fig_electricity_sea_volume.show()

In [14]:
fig_electricity_sea_portion = go.Figure()
fig_electricity_sea_portion = fig_electricity_sea_portion.add_trace(go.Bar(x = electricity_sea_sum.index, y = electricity_sea_sum['% Fossil Fuel'], name = 'Fossil Fuel'))
fig_electricity_sea_portion = fig_electricity_sea_portion.add_trace(go.Bar(x = electricity_sea_sum.index, y = electricity_sea_sum['% Nuclear'], name = 'Nuclear'))
fig_electricity_sea_portion = fig_electricity_sea_portion.add_trace(go.Bar(x = electricity_sea_sum.index, y = electricity_sea_sum['% Renewable'], name = 'Renewable'))
fig_electricity_sea_portion = fig_electricity_sea_portion.update_layout( title = 'Electricity Consumption Mix in SEA', xaxis_tickfont_size = 7, template = 'plotly_dark', barmode = 'stack')
fig_electricity_sea_portion.show()

Total electricity consumption throughout the region shows an increasing trend throughout 2000 to 2019, before collapsing in the year 2020 due to Covid-19 outbreak. In 2000, the region produced 336.07 TWh and by 2005, it increased to 452 TWh, an increase of 34.5% throughout the region. By the of the decade, its consumption increased by 74.37% from early decade to stand at 568.61 TWh.

In this early decade, electricity consumption in the region is driven by fossil fuel consumption. Fossil fuel formed more than 80% of the electricity consumed in the region. In 2000, 281 TWh or 83.61% of the electricity consumed was generated by fossil fuel. The number keeps increasing. By 2005, it increased by 40.17% to 393.89 TWh. The volume is 87.14% of total electricity consumption in the region. By the end of the decade, 494.27 TWh of fossil fuel was consumed. However, fossil fuels mix of total electricity consumption slightly decreased to 86.93% due to the increase in renewable energy consumption.  

Renewable energy in the region plays crucial roles in the region. In 2000, 16.39% or 55.07 TWh of renewable energy being consumed. It grew slightly by 5.56% to 58.13 by 2005 TWh. However, renewable energy shares in total electricity consumption mix fell slightly to 12.86 TWh. In 2010, its consumption is at 74.34 TWh, an increase of 35% from the early decade. The volume makes up 13.07% of total electricity consumption in the region.  

In the second decade, total electricity consumption in the region keeps increasing. By 2015, it increased by 24.93% to 710.36 TWh and grow further to reach its peak at 854.5 TWh in 2019. Covid-19 outbreak in 2020 however brought down total electricity consumption by 3.37% to 825.74 TWh in 2020.  

In the decade between 2010 to 2020, renewable energy grew at a faster rate than fossil fuels. Between 2010 and 2015, renewable energy consumption increased by 28.58% to 95.59 TWh while fossil fuels grew at slightly lower rates, 24.37% to 614.77 TWh. The period between 2015 to 2020 saw renewable energy consumption grew rapidly at 54.90% to reach its peak at 148.15 TWh in 2020. The largest increase was recorded between 2015 to 2018 when its consumption grew by 51.20% from 95.59 TWh in 2020 to 144.54 TWh in 2018. Fossil fuels consumption meanwhile grew at a much slower rate as it increased by 15.6% to its peak at 710.66 TWh in 2019. Fossil fuels consumption declined in 2020 due to pandemic outbreak to 677.59 TWh. In contrast, renewable energy consumption was not affected by the pandemic outbreak as its consumption reached its peak in 2020.  

Faster renewable energy consumption growth led to the increase of its share in total electricity consumption mix, particularly between 2015 to 2020. In 2010, it contributed to 13.07% of total electricity consumption. By 2015, its shares increased slightly to 13.46% in 2015. Rapid increase in renewable energy consumption led to an increase in its share in total electricity consumption. By 2020, it represented 17.94% of regional electricity consumption.  


In [15]:
electricity_sea_country_sum= electricity_sea[['Year','Entity','Electricity from fossil fuels (TWh)','Electricity from renewables (TWh)']]
electricity_sea_country_sum.fillna(0,inplace=True)
electricity_sea_country_sum['Total Electricity'] = electricity_sea_country_sum['Electricity from fossil fuels (TWh)'] + electricity_sea_country_sum['Electricity from renewables (TWh)']
electricity_sea_country_sum.head()


,Year,Entity,Electricity from fossil fuels (TWh),Electricity from renewables (TWh),Total Electricity
567,2000,Cambodia,0.42,0.05,0.47
568,2001,Cambodia,0.48,0.03,0.51
569,2002,Cambodia,0.66,0.03,0.69
570,2003,Cambodia,0.67,0.04,0.71
571,2004,Cambodia,0.73,0.03,0.76


In [16]:
electricity_sea_country_portion = electricity_sea_country_sum[['Year','Entity','Electricity from renewables (TWh)','Electricity from fossil fuels (TWh)','Total Electricity']].reset_index(drop=True)
electricity_sea_country_portion['portion_renewable'] = (electricity_sea_country_portion['Electricity from renewables (TWh)']/electricity_sea_country_portion['Total Electricity'])*100
electricity_sea_country_portion['portion_fossil'] = (electricity_sea_country_portion['Electricity from fossil fuels (TWh)']/electricity_sea_country_portion['Total Electricity'])*100

electricity_sea_country_portion.head()

,Year,Entity,Electricity from renewables (TWh),Electricity from fossil fuels (TWh),Total Electricity,portion_renewable,portion_fossil
0,2000,Cambodia,0.05,0.42,0.47,10.638298,89.361702
1,2001,Cambodia,0.03,0.48,0.51,5.882353,94.117647
2,2002,Cambodia,0.03,0.66,0.69,4.347826,95.652174
3,2003,Cambodia,0.04,0.67,0.71,5.633803,94.366197
4,2004,Cambodia,0.03,0.73,0.76,3.947368,96.052632


In [17]:
def draw_graph(countries,column,data_frame):
    fig_electricity_sea_country_volume = go.Figure()
    
    for country in countries:
        country_data = data_frame[data_frame['Entity'] == country]
        fig_electricity_sea_country_volume.add_trace(go.Scatter(x = country_data['Year'], y = country_data[column], name = country, mode = 'lines+markers'))
    fig_electricity_sea_country_volume.update_layout( title = 'Electricity Consumption in SEA' + ' : ' + column, xaxis_tickfont_size = 7, template = 'plotly_dark')
    fig_electricity_sea_country_volume.show()

## **Total Electricity Consumption & Source of Electricity Consumption**

In [18]:
draw_graph(sea,'Total Electricity',electricity_sea_country_sum)

In [19]:
draw_graph(sea,'Electricity from fossil fuels (TWh)',electricity_sea_country_sum)

In [20]:
draw_graph(sea,'Electricity from renewables (TWh)',electricity_sea_country_sum)

Between 2000 to 2005, Indonesia is the largest electricity consumer in the region. Its electricity consumption increased from 98.03 TWh to 132.88 TWh, an increase of 35.56% within five years. However, Thailand the second largest electricity consumer in the region grew at faster rate, increased from 89.53 TWh to 123 TWh, an increase of 37% within 5 years. Total electricity consumption in Malaysia grew at even faster rate. It grew at 39.62% within 5 years, from 70.47 TWh to 98.39 TWh.  

Indonesia, Malaysia, and Thailand remained three largest electricity consumers in the region. All three countries showed a similar uptrend throughout the second decade of 2010 to 2020, except for 2020. However, total electricity consumption in Indonesia increased at faster rates compared with Malaysia and Thailand. Between 2010 to 2015, total electricity consumption in Indonesia increased by 37.05% to 243.27 TWh. In comparison, electricity consumption in Malaysia increased by 10.93% and Thailand by 20.3% between the same period. By 2019, total electricity consumption in Indonesia increased by 21.07% to its peak at 295.43 TWh. Meanwhile, total electricity consumption in Malaysia and Thailand grew by 14.32% and 18.89% respectively.  

Rapid increases in the total electricity consumption in Indonesia was fulfilled by the increase in consumption of fossil fuels and renewable energy. Between 2010 to 2015, fossil fuels were the main driver for total electricity consumption. Fossil fuels consumption within this period grew by 46.77%, from 142.88 TWh to 209.71 TWh. Within this period, renewable energy consumption declined by 3.09%, from 34.63 TWh to 33.56 TWh. Renewable energy consumption in Indonesia increased rapidly between 2015 to 2018, increased by 44.16%, from 33.56 TWh to 48.38 TWh and further increased to 52.91 TWh in 2020, defying the decline in the overall electricity consumption due to the outbreak of Covid-19. Between 2015 to 2019, fossil fuel consumption in Indonesia increased by 17.97% to reach its peak, 247.39 TWh in 2019.  


In [21]:
def draw_graph_bar(countries,column1,column2,dataframe):
    for country in countries:
        country_data = dataframe[dataframe['Entity'] == country]
        fig_electricity_sea_country_volume = go.Figure()
        fig_electricity_sea_country_volume.add_trace(go.Bar(x = country_data['Year'], y = country_data[column1], name = 'Renewable Energy'))
        fig_electricity_sea_country_volume.add_trace(go.Bar(x = country_data['Year'], y = country_data[column2], name = 'Fossil Energy'))
        fig_electricity_sea_country_volume.update_layout( title = 'Electricity Consumption Mix in SEA' + ' : ' + country, xaxis_tickfont_size = 7, template = 'plotly_dark', barmode = 'stack')
        fig_electricity_sea_country_volume.show()

## **Electricity Consumption Mix**

In [22]:
draw_graph_bar(sea,'portion_renewable','portion_fossil',electricity_sea_country_portion)

In most Southeast Asian countries, fossil fuels are the main source of electricity consumption, with the exception of Cambodia and Myanmar. 

In Cambodia, renewable energy initially contributed a small share of electricity consumption, ranging from 3.95% to 10.64% between 2000 and 2005. However, the share of renewable energy in electricity consumption increased significantly in the following years. By 2010, renewable energy accounted for 12.9% of total electricity consumption in Cambodia. In 2013, the share of renewable energy peaked at 72.83%, and by the end of the decade, more than half of total electricity consumption in Cambodia was derived from renewable energy. 

Renewable energy has played a significant role in Myanmar's electricity sector. Throughout the period from 2000 to 2020, more than 40% of the total electricity consumption in Myanmar was derived from renewable energy. Between 2005 and 2018, renewable energy accounted for more than half of electricity consumption in the country. In 2009 and 2011, almost 80% of the total electricity consumption was generated from renewable energy. 

The contribution of renewable energy to electricity consumption in other Southeast Asian countries is low, but it is generally improving. This is especially true for the major electricity consumers in the region, Malaysia, Thailand, and Indonesia. By the end of the period, all of these major electricity consumers had recorded an increase in the share of renewable energy in their electricity mix. 

# **Primary Energy Consumption Analysis**

Primary energy is energy that is extracted directly from its sources, before any transformation or conversion. The energy that is available in its raw or unprocessed form is used to power various sectors and industry.  

In [23]:
primary_energy_sea = data_sea[['Entity','Year','Primary energy consumption per capita (kWh/person)','Energy intensity level of primary energy (MJ/$2017 PPP GDP)','Renewables (% equivalent primary energy)','gdp_per_capita']]
primary_energy_sea.fillna(0,inplace=True)
primary_energy_sea.head()

,Entity,Year,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Renewables (% equivalent primary energy),gdp_per_capita
567,Cambodia,2000,739.45550,7.91,0.0,300.613679
568,Cambodia,2001,769.40970,7.51,0.0,321.150224
569,Cambodia,2002,754.14470,7.05,0.0,338.987477
570,Cambodia,2003,803.81635,6.69,0.0,362.335482
571,Cambodia,2004,813.56445,6.23,0.0,408.513639


## **Primary Energy Consumption per Capita**

In [24]:
def draw_primary_graph(countries,column,data_frame):
    fig_primary_energy_sea_country_volume = go.Figure()
    
    for country in countries:
        country_data = data_frame[data_frame['Entity'] == country]
        fig_primary_energy_sea_country_volume.add_trace(go.Scatter(x = country_data['Year'], y = country_data[column], name = country, mode = 'lines+markers'))
    fig_primary_energy_sea_country_volume.update_layout( title = 'Primary Energy Consumption in SEA' + ' : ' + column, xaxis_tickfont_size = 7, template = 'plotly_dark')
    fig_primary_energy_sea_country_volume.show()

draw_primary_graph(sea,'Primary energy consumption per capita (kWh/person)',primary_energy_sea)


Singapore recorded the highest primary energy consumption per capita throughout the period between 2000 to 2020. The largest increase in the consumption of primary energy per capita was between 2005 to 2011. In 2005, primary energy consumption per capita for Singapore stood at 123.55 kWh and by 2011, it increased by 23% to 151.96 kWh. However, primary energy consumption per capita experienced a second decline between 2011 to 2014. In the first fall, primary energy consumption per capita fell from its peak at 121.49 kWh in 2001 to 107.49 kWh in 2003, a decline of 11.52%. Between 2011 to 2015, primary energy consumption per capita declined by 2.15% before reaching its peak in 2017 at 164.80 kWh. Primary energy consumption per capita declined by 6.4% afterwards.  

Similar trend also displayed by Malaysia. Primary energy consumption per capita increased rapidly between 2000 to 2006. Primary energy consumption per capita increased by 20.91% from 26.64 kWh to 32.21 kWh. However, between 2008 to 2011, primary energy consumption per capita for Malaysia decreased to the lowest level in 4 years to 32 kWh. Between 2009 to 2011, primary energy consumption per capita lingered around 32 kWh before increasing to 36.06 kWh in 2013. It stayed flat between 2013 to 2015. Primary energy consumption per capita increased again to reach its peak in 2019 to 37.82 kWh before decreasing to 36 kWh due to the outbreak of Covid-19 pandemic.  

Indonesia also displayed similar trend like Singapore and Malaysia. Primary energy consumption increased from 5.44 kWh to 6.22 kWh between 2000 to 2003. Primary energy consumption per capita declined to 6.087 kWh by 2004. Primary energy consumption per capita increased from 6.19 kWh in 2005 to reach its highest level in 12 years to 7.57 kWh, an increase of 22.29%. It declined again by 4.49% to 7.23 kWh in 2013 and remain flat for 3 years. Primary energy consumption per capita increased to reach its highest level at 8.47 kWh in 2019 before affected by the outbreak of Covid-19.  

On the contrary, primary energy consumption per capita for Thailand displayed a continuous upward trend from 2000 to 2019. In the first decade, from 2000 to 2010, primary energy consumption per capita for Thailand increased by 47.12%, from 11.76 kWh to 17.3 kWh. Between 2010 to 2019, primary energy consumption per capita increased by 20.17% to 20.79 kWh before declining by 7.17% to 19.3 kWh.  

Similarly, Cambodia also shown similar upward trend. Between 2000 to 2010, primary energy consumption increased by more than two times, from 0.74 kWh to 1.59 kWh. In the next decade, primary energy consumption per capita for Cambodia increased at more impressive rates. By 2019, primary energy consumption per capita increased by 2.5 times, to 4.07 kWh.

In general, primary energy consumption per capita in the region was on an upward trend from 2000 to 2019. In 2020, the region endured most of the outbreak of Covid-19 pandemic, leading to the decline in primary energy consumption per capita.  

In [25]:
def double_y_axes_consumption_gdp(countries,column_1,column_2,dataframe):
    for country in countries:
        country_data = dataframe[dataframe['Entity'] == country]
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column_1], name=f'{country} - {column_1}', mode='lines+markers'), secondary_y=False)
        fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column_2], name=f'{country} - {column_2}', mode='lines+markers'), secondary_y=True)
        fig.update_layout(title=f'Primary Energy Consumption & GDP - {country}',xaxis_title='Year',yaxis_title=column_1,template='plotly_dark')
        fig.update_yaxes(title_text=column_1, secondary_y=False)
        fig.update_yaxes(title_text=column_2, secondary_y=True)
        fig.update_xaxes(tickfont=dict(size=10), title_font=dict(size=10))
        fig.update_yaxes(tickfont = dict(size=10), title_font = dict(size=10))
        fig.show()

## **Primary Energy Consumption & Gross Domestic Product (GDP)**

In [26]:
double_y_axes_consumption_gdp(sea,'Primary energy consumption per capita (kWh/person)','gdp_per_capita',primary_energy_sea)

The rise in per capita primary energy consumption across the region can be attributed to its concurrent GDP growth. Over the period from 2000 to 2019, there was a consistent increase in GDP per capita throughout the region. This economic growth corresponded to an increase in per capita primary energy consumption.  

## **Primary Energy Intensity**

In [27]:
draw_primary_graph(sea,'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',primary_energy_sea)

Despite being the highest primary energy consumer per capita, energy consumption in Singapore is much more efficient than the rest of the region. In general, energy consumption efficiency as measured by energy intensity level for Singapore is declining, showing that its energy consumption become much efficient. Between 2000 to 2005, energy intensity for Singapore declined by 11.27% from 3.46 to 3.07, despite temporary hike between 2002 to 2005. Energy intensity declined further by 19.21% to 2.48 in 2010.  

On the contrary, Malaysia, the second largest primary energy consumer per capita is less efficient compared with Singapore, Indonesia, and Thailand. Despite this, Malaysia recorded improved energy efficiency throughout 2000 to 2022. Between 2000 to 2005, energy intensity in Malaysia remained flat. Energy intensity in Malaysia during this period ranged between 5. 48 to 5.85. Energy intensity between 2005 to 2012 recorded further improvement in Malaysia’s energy efficiency. Energy intensity declined by 14.7% to 4.99 in 2012. By 2019, energy intensity in Malaysia decreased further to its lowest level, 4.25. Throughout the period, energy intensity declines by 22.45% in 19 years.  

For Thailand, its energy intensity increased from 4.93 in 2000 to reach its peak, 5.28 in 2013, an increase of 7.1%. Afterwards, Thailand improves its energy efficiency. Its energy intensity continues to decline and reached its lowest by 2018, 4.5, a decline of 14.77%, reflecting a continuous improvement towards its energy efficiency.  

In comparison with other major primary energy consumer in the region, Indonesia recorded a remarkable improvement in its energy efficiency. Between 2000 to 2019, its energy intensity dropped from 5.42 to 3.16, a decrease of 41.7%. Remarkable improvements in its energy intensity were recorded between 2000 to 2016 as energy intensity declined by 41.14% to 3.19.

In [28]:
primary_re_energy_sea = primary_energy_sea[['Entity','Year','Renewables (% equivalent primary energy)']]
primary_re_energy_sea

,Entity,Year,Renewables (% equivalent primary energy)
567,Cambodia,2000,0.000000
568,Cambodia,2001,0.000000
569,Cambodia,2002,0.000000
570,Cambodia,2003,0.000000
571,Cambodia,2004,0.000000
...,...,...,...
3266,Thailand,2016,4.599976
3267,Thailand,2017,5.306326
3268,Thailand,2018,6.380221
3269,Thailand,2019,6.956220


In [29]:
def primary_energy_portion_sea(countries,column,dataframe):
    fig_re_sea_portion = go.Figure()
    for country in countries:
        country_data = dataframe[dataframe['Entity'] == country]
        fig_re_sea_portion.add_trace(go.Scatter(x = country_data['Year'], y = country_data[column], name = country, mode = 'lines+markers'))
    fig_re_sea_portion.update_layout( title = 'Renewable Energy in Primary Energy Consumption in SEA (%)', xaxis_tickfont_size = 7, template = 'plotly_dark')
    fig_re_sea_portion.show()

## **Renewable Energy in Primary Energy Consumption**

In [30]:
primary_energy_portion_sea(sea,'Renewables (% equivalent primary energy)',primary_re_energy_sea)

Except for the Philippines, renewable energy mix in primary energy consumption increased throughout the period.  

For Indonesia, renewable energy consumption increased from meagre 3.22% in 2003 to almost 10% in 2020, with rapid increases between 2015 and 2020. In 2015, renewable energy consumption contribution stood at 3.79% before increasing to 7.87% in 2019. By 2020, the contribution stood at 9.86%.  

For Malaysia, renewable energy consumption as part of primary energy consumption fell initially between 2000 and 2007 to record the lowest contribution of 2.14%. Afterwards, it increased to 3.65% in 2015 in Malaysia. A big jump in contribution of renewable energy as part of total primary energy consumption was recorded between 2015 and 2017. In 2017, renewable energy contributed to 6.29% and reached its peak by the end of the decade. In 2020, during Covid-19 outbreak, renewable energy contributed 7.16% of primary energy consumption.

Similarly, for Thailand, renewable energy share in primary energy consumption fell initially to 2.28% in 2005. Afterwards, the share increased further to 2.8% in 2010. Between 2010 to 2020, renewable energy share in primary energy consumption grew rapidly as compared with previous decade. In 2015, its shares stood at 4.19%. By 2020, the share stood at 6.99% of primary energy consumption in Thailand.

On the contrary, Philippines recorded defied the upward trend of its neighbors. Shares of renewable energy in primary energy consumption declined significantly, especially between 2012 and 2019. In 2000, 18.6% of primary energy consumption derived from renewable energy. By 2010, it declined to 14.19%. Between 2012 to 2019, renewable energy shares further declined to their lowest level to 10.23%. 

# **CO2 Emission Analysis**

In [31]:
co2_sea = data_sea[['Entity','Year','Value_co2_emissions_kt_by_country','gdp_growth',]]
co2_sea.head()

,Entity,Year,Value_co2_emissions_kt_by_country,gdp_growth
567,Cambodia,2000,1960.000000,9.993580
568,Cambodia,2001,2150.000000,8.148386
569,Cambodia,2002,2210.000038,6.578940
570,Cambodia,2003,2380.000114,8.505896
571,Cambodia,2004,2380.000114,10.340529


In [32]:
co2_sea_sum = pd.DataFrame(co2_sea.groupby('Year')['Value_co2_emissions_kt_by_country'].sum()).reset_index()
co2_sea_sum

,Year,Value_co2_emissions_kt_by_country
0,2000,6.951200e+05
1,2001,7.276900e+05
2,2002,7.496300e+05
3,2003,7.949700e+05
4,2004,8.369900e+05
5,2005,8.542900e+05
6,2006,8.780200e+05
7,2007,9.200500e+05
8,2008,9.321600e+05
9,2009,9.206400e+05


## **Total Regional CO2 Emission**

In [33]:
fig_co2_sea_sum = go.Figure()
fig_co2_sea_sum = fig_co2_sea_sum.add_trace(go.Scatter(x = co2_sea_sum['Year'], y = co2_sea_sum['Value_co2_emissions_kt_by_country'], name = 'CO2 Emissions', mode = 'lines+markers'))
fig_co2_sea_sum.update_layout( title = 'CO2 Emissions in SEA', xaxis_tickfont_size = 7, template = 'plotly_dark')
fig_co2_sea_sum.show()

CO2 emissions in Southeast Asia increased steadily from 2000 to 2019, with the fastest growth between 2000 and 2005. In that period, emissions grew by 22.9%. The growth rate slowed in the following years, but emissions still increased by 15.62% from 2005 to 2010 and 18.81% from 2010 to 2015. The fastest growth in recent years was between 2016 and 2019, when emissions increased by 17.34%. 

In [34]:
def  co2_sea_sum_country(countries,column,dataframe):
    fig_co2_sea_sum_country = go.Figure()
    for country in countries:
        country_data = dataframe[dataframe['Entity'] == country]
        fig_co2_sea_sum_country = fig_co2_sea_sum_country.add_trace(go.Scatter(x = country_data['Year'], y = country_data[column], name = country, mode = 'lines+markers'))
    fig_co2_sea_sum_country.update_layout( title = 'CO2 Emissions in SEA' , xaxis_tickfont_size = 7, template = 'plotly_dark')
    fig_co2_sea_sum_country.show()

# **CO2 Emission & Gross Domestic Product (GDP)**

In [35]:
co2_sea_sum_country(sea,'Value_co2_emissions_kt_by_country',co2_sea)

Indonesia is the largest CO2 emitter in the region. In 2000, 0.28065 Mt and by 2005, it increased by 21.91%, to 0.34215 Mt. Between 2005 and 2010, CO2 emissions increased by 21.44%, to 0.41552 Mt. Increased in CO2 emissions slowed down between 2010 to 2015, as it increased by 17.58% to 0.48855 Mt. However, the trend was reversed in 2016 as CO2 emissions increased by 28.46% from 0.48251 Mt to 0.61984 Mt in 2019.  

CO2 emissions in Thailand showed an increasing trend from 2000 to 2019. The fastest growth was between 2000 and 2005, when emissions increased by 35.13%. The growth rate slowed in the following years, but emissions still increased by 7.63% from 2005 to 2010 and 12.64% from 2010 to 2015. A slight decline was seen between 2015 and 2018, but emissions increased again in 2019 to their highest level in 19 years. 

CO2 emissions in Malaysia showed an increasing trend from 2000 to 2019. The fastest growth was between 2000 and 2008, when emissions increased by 63.16%. However, emissions fell in 2009 to 0.18193 Mt. By the end of the decade, emissions had rebounded to 0.20022 Mt, but were still below the peak level of 2008. Emissions continued to increase in 2015 and 2019, reaching 0.23654 Mt and 0.25327 Mt, respectively. 

Despite being the highest primary energy consumer in Southeast Asia, Singapore's CO2 emissions were relatively low, ranging from 42.12 kt to 47.38 kt from 2000 to 2019. The peak emission was 49.14 kt in 2017. 

In contrast, the Philippines, with lower primary energy consumption, had higher CO2 emissions. Between 2000 and 2005, CO2 emissions in the Philippines ranged from 72.1 kt to 76.67 kt. However, emissions declined to 67.47 kt in 2006. After that, emissions trended upwards and reached a peak of 145.42 Mt in 2019. This represents a two-fold increase in CO2 emissions over two decades. 

## **CO2 Emission vs GDP**

In [36]:
def co2_gdp_sea(countries,column_1,column_2,dataframe):
    for country in countries:
        country_data = dataframe[dataframe['Entity'] == country]
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column_1], name=f'{country} - C02 Emission', mode='lines+markers'), secondary_y=False)
        fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column_2], name=f'{country} - GDP Growth', mode='lines+markers'), secondary_y=True)
        fig.update_layout(title=f'CO2 Emissions vs GDP - {country}',xaxis_title='Year',yaxis_title=column_1,template='plotly_dark')
        fig.update_yaxes(title_text=column_1, secondary_y=False)
        fig.update_yaxes(title_text=column_2, secondary_y=True)
        fig.update_xaxes(tickfont=dict(size=10), title_font=dict(size=10))
        fig.update_yaxes(tickfont = dict(size=10), title_font = dict(size=10))
        fig.show()

co2_gdp_sea(sea,'Value_co2_emissions_kt_by_country','gdp_growth',co2_sea)

From 2000 to 2019, all countries in Southeast Asia recorded positive GDP growth. This was driven by increased economic activities, which led to increased CO2 emissions in all countries. The COVID-19 pandemic caused a contraction in economic activities in all countries except Myanmar. However, CO2 emissions in all countries except Singapore still reached a peak in 2020. This is due to the region's reliance on fossil fuels, as evidenced by the high primary energy consumption in all countries. The pandemic did slow down the rate of increase in CO2 emissions, but it did not prevent emissions from reaching a peak. 

# **Conclusion**

The demand for energy in Southeast Asia is growing rapidly, driven by the region's rapid economic growth. This increasing demand is putting a strain on the environment, as the region relies heavily on fossil fuels to meet its energy needs.

However, there are some positive developments that could help to mitigate the environmental impact of energy demand in Southeast Asia. These include:

* The increasing use of renewable energy: Renewable energy sources, such as solar and wind power, do not produce greenhouse gases, so they  can help to reduce air pollution and climate change.
* The increasing energy efficiency: Energy efficiency measures, such as improving insulation and using energy-efficient appliances, can help to reduce the amount of energy needed to power homes and businesses.

If these developments continue, Southeast Asia will be able to lessen its dependency on fossil fuels in the medium and long term, ensuring a more sustainable future for the region.